# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [34]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama


In [35]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'


In [36]:
# set up environment
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [53]:
# here is the question; type over this to ask something new

myQuestion = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""


In [38]:
# Define our system prompt
system_prompt = """You are an assistant that analyzes the code and explains it in a clear and easy-to-understand manner.
Example: print("Hello world")
Respond in markdown: Sure! Let's break it down step by step:
print("Hello world")
This is a simple Python statement that does one thing: it shows the text "Hello world" on the screen.
1. print:
The word print is a command in Python.
It tells the computer: 'Display something for me.'
2. "Hello world":
The words "Hello world" are inside quotes.
Anything inside quotes is called a string in Python, which means it's just text.
Together:
The print command looks at what's inside the parentheses ( ) and displays it on the screen.
In this case, it displays the words Hello world.
Output:
When you run the code, the computer will show:
Hello world
It’s like saying to the computer:
"Hey, computer, please write 'Hello world' so people can see it!" 😊"""

In [39]:
def get_user_prompt(question):
    user_prompt = f"Explain the question {question}\n"
    user_prompt += "In a clear and easy to understand manner in markdown.\n"
    return user_prompt

In [40]:
print(get_user_prompt(question))

Explain the question 
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}

In a clear and easy to understand manner in markdown.



In [45]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()
def answer_question_with_streaming_OpenAI(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(question, )}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [42]:
answer_question_with_streaming(question)

Sure! Let's break down this code step by step:

python
yield from {book.get("author") for book in books if book.get("author")}


This line of code is a generator expression that yields unique authors from a collection of books. Here's a detailed explanation:

1. **yield from**:
   - The keyword `yield` is used in Python to create a generator, which can be paused and resumed, returning values one at a time.
   - The `from` keyword indicates that you want to yield items from an iterable (like a list or a set). In this case, it’s yielding from a set comprehension.

2. **{...}**:
   - The curly braces `{}` indicate that we are creating a set. A set is a collection of unique items in Python, which means it will automatically remove any duplicate entries.

3. **book.get("author")**:
   - This part accesses the value associated with the key "author" in each `book` dictionary. Here, `book` represents each individual item in the `books` collection.
   - The `get` method is used for dictionaries to safely retrieve the value for a given key. If the key doesn't exist, it will return `None` instead of throwing an error.

4. **for book in books**:
   - This part of the code iterates over each `book` in a collection called `books`. It assumes that `books` is a list (or any iterable) of dictionaries.

5. **if book.get("author")**:
   - This is a condition that filters the books. It ensures that only books with a valid "author" (i.e., not `None` or an empty string) are included in the set comprehension. If `book.get("author")` returns a valid author, the `if` clause evaluates to `True`.

### Together:

- The entire line of code creates a set of unique authors from the list of books, while ignoring any books that do not have an author specified.
- The `yield from` statement then yields each author one at a time, allowing you to iterate over them or collect them in another structure without creating a full list in memory.

### Example:

If you have the following books:
python
books = [
    {"title": "Book One", "author": "Alice"},
    {"title": "Book Two", "author": "Bob"},
    {"title": "Book Three", "author": "Alice"},
    {"title": "Book Four", "author": None},
]


When you run the line of code, the output will yield:

Alice
Bob

Output:
- The generator will yield unique authors, which means even though "Alice" is listed twice, it will only show up once.

### Conclusion:

Essentially, this code is a convenient way to extract a unique list of authors from a dataset of books, making it easy to work with authors throughout your program. It performs this function efficiently without needing extra memory for duplicates. 😊

In [64]:
# Get Llama 3.2 to answer

def answer_question_with_streaming_Ollama(question):
    print(question)
    stream = ollama.chat(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(question)}
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    chunk_accumulator = []  # To batch small chunks
    for chunk in stream:
        # Debug: print the full chunk
        #print(chunk)

        if 'message' in chunk and 'content' in chunk['message']:
            content = chunk['message']['content']
            chunk_accumulator.append(content)

            # Accumulate the response
            response += content

            # Update the display periodically (e.g., every 5 chunks)
            if len(chunk_accumulator) >= 5:
                display_content = response.replace("```", "").replace("markdown", "")
                update_display(Markdown(display_content), display_id=display_handle.display_id)
                chunk_accumulator = []  # Reset the accumulator

    # Final update to display any remaining chunks
    display_content = response.replace("```", "").replace("markdown", "")
    update_display(Markdown(display_content), display_id=display_handle.display_id)


In [65]:
answer_question_with_streaming_Ollama(myQuestion)


Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}



Let's break down this complex code step by step:

### Understanding the Code
python
yield from {book.get("author") for book in books if book.get("author")}

This line of code is using several advanced features of Python, so let's break it down into smaller parts.

**1. `yield from`**
The word `yield` is used to create a generator function in Python.
A generator function is like a normal function, but instead of returning all the values at once (like a regular list), it returns them one by one.
When you use `yield`, the function doesn't finish executing until it reaches the `yield` keyword.

**2. `{}`**
These curly brackets are used to create an expression in Python called a **dictionary comprehension**.
A dictionary comprehension is like a regular dictionary, but instead of using the `dict()` function or `{key: value}`, you use this syntax.

**3. `.get("author") for book in books`**
This part of the code is using another advanced feature called a **generator expression**.
It's similar to a list comprehension, but instead of creating a new list, it creates an iterator that yields each value one by one.

* `.get("author")`: This method calls the `get()` function on the `book` object and returns the value associated with the key `"author"`.
* `for book in books`: This part loops over each item (`book`) in a collection (`books`).

So, this generator expression takes each book in the `books` list, gets its author (if it exists), and yields that author.

**4. `yield from {...}`**
When you use `yield from`, it's like saying "yield all these values, one by one".

In this case, the dictionary comprehension `{book.get("author") for book in books if book.get("author")}` is yielding a dictionary with authors as values.
By using `yield from`, we're essentially saying "yield each author, one by one".

**Putting it All Together**
So, what does this code do?
It creates an iterator that yields each author's name from the `books` list, one by one. The resulting iterator is like a stream of authors.

Here's an example:
python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3"}
]

for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)

Output:

Author A
Author B

Note that this code uses a clever way to iterate over the authors, but it's still just printing each author's name one by one.